In [ ]:
###用于测试csv和mlp
import torch
import torchvision.models as models
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim

import time
from collections import namedtuple

from torch.utils.data.sampler import WeightedRandomSampler

import torch.nn as nn
import csv
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# 五层神经网络
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 200)
        self.fc3 = nn.Linear(200, 100)
        self.fc4 = nn.Linear(100, num_classes)
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out

mlp = NeuralNet(14,200,4)
print(mlp)

##############################################读写CSV
file1 = "./trainData/dataAllSim.csv"
##方式1
#data1=[]
#csv_reader = csv.reader(open(file1))
#for row in csv_reader:
#        data1.append(row)
#print(data1[0])        

##方式2
#https://blog.csdn.net/zw__chen/article/details/82806900
csvfile = open(file1, 'r')
xyData= np.loadtxt(file1,delimiter=',',skiprows=1)
x = xyData[:,0:-1]
y = xyData[:,-1]
print(y.shape )
print(x.shape )
x1 =torch.from_numpy(x)
y1 =torch.from_numpy(y)

dataset1 = TensorDataset(x1.float(),y1.long())

#train_dataset, test_dataset = random_split(
#    dataset=dataset1,
#    lengths=[7, 3],
#    generator=torch.Generator().manual_seed(0)
#)

#################################
#数据准备：平衡
tmp = y
trainratio = np.bincount(tmp.astype("int64"))
train_weights = 1./trainratio
train_weights[0]= train_weights[0]*2
train_sampleweights = torch.from_numpy(train_weights)
train_sampler = WeightedRandomSampler(weights=train_sampleweights, num_samples = len(y))
#train_loader = DataLoader( dataset1, batch_size=5120, num_workers=1, sampler=train_sampler)


train_loader = DataLoader(dataset=dataset1, batch_size=5120, shuffle=True)

#准备设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == 'cuda':
    mlp = torch.nn.DataParallel(mlp)
    cudnn.benchmark = True
    mlp =  mlp.to(device)
print(device)

#准备模型
modelPathName = "./trainedModes/"+"redTLS1.modeparams"
params = torch.load(modelPathName)
mlp.load_state_dict(params['net'])


#################################
#训练开始
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
running_loss = 0
startEpoch = 0
epochs = 3000#大概8小时
print("start training")

for epoch in range(startEpoch, epochs):  # 多批次循环

    mlp.train()
    time_start = time.time()
    total = 0
    correct = 0
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        # 获取输入
        #print(epoch,i)
        inputs, labels = data
        #inputs, labels = inputs.to(device), labels.to(device)
        inputs=inputs.type(torch.float32)
        labels=labels

        if device == 'cuda':
            inputs, labels = inputs.to(device), labels.to(device)

        # 梯度置0
        optimizer.zero_grad()

        # 正向传播，反向传播，优化a
        outputs = mlp(inputs)
        #print("inputs:",inputs)
        #print("###################################")
        #print("outputs:",outputs)
        #print("labels",labels)

        loss = criterion(outputs, labels)
        #print("loss:",loss.item())

        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        #print("labels.size(0):",labels.size(0))
        #print("predicted:",predicted)
        #print("predicted.eq(labels).sum().item():",predicted.eq(labels).sum().item())

        running_loss += loss.item()

        #os.input()
        if i % 20 == 1:  # 每200批次打印一次
         time_end = time.time()
         print('##########one 20 batch totally time cost %.3f' %(time_end-time_start))
         print("batchIndex %d |trainLen %d | Loss: %.3f | Acc: %.3f | correct, total: (%d,%d)" % (
             i, len(train_loader), running_loss/(i+1), 100.*correct/total, correct, total))

    time_end = time.time()
    

    state = {"net": mlp.state_dict(), "optimizer": optimizer.state_dict()}
    modelPathName = "./trainedModes/"+"redTLS.modeparams"
    torch.save(state, modelPathName)
    params = torch.load(modelPathName)
    mlp.load_state_dict(params['net'])
    optimizer.load_state_dict(params['optimizer'])
    
    #用全训练集合进行评估
    mlp.eval()
    labels = y1.long()
    outputs = mlp(x1.float())
    _, predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()
    
    print("\n\n Eval--epochIndex %d |time: %d | Acc: %.3f | correct, total: (%d,%d)" % (
         epoch, (time_end-time_start), 100.*correct/total, correct, total))
    tmp1 = classification_report(labels,predicted)
    tmp2 = confusion_matrix(labels,predicted)
    print(tmp1)
    print(tmp2)
  
    print("########################################################################")


NeuralNet(
  (fc1): Linear(in_features=14, out_features=200, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=4, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
(723532,)
(723532, 14)
cpu
start training
##########one 20 batch totally time cost 0.455
batchIndex 1 |trainLen 142 | Loss: 0.179 | Acc: 92.539 | correct, total: (9476,10240)
##########one 20 batch totally time cost 3.645
batchIndex 21 |trainLen 142 | Loss: 0.170 | Acc: 92.964 | correct, total: (104715,112640)
##########one 20 batch totally time cost 6.833
batchIndex 41 |trainLen 142 | Loss: 0.168 | Acc: 93.028 | correct, total: (200047,215040)
##########one 20 batch totally time cost 10.106
batchIndex 61 |trainLen 142 | Loss: 0.167 | Acc: 93.066 | correct, total: (295430,317440)
##########one 20 batch totally time cost 13.361
batchIndex 81 |trainLen 142 | Loss: 0.166 | Acc

##########one 20 batch totally time cost 17.143
batchIndex 101 |trainLen 142 | Loss: 0.155 | Acc: 93.404 | correct, total: (487795,522240)
##########one 20 batch totally time cost 20.430
batchIndex 121 |trainLen 142 | Loss: 0.154 | Acc: 93.427 | correct, total: (583581,624640)
##########one 20 batch totally time cost 23.646
batchIndex 141 |trainLen 142 | Loss: 0.156 | Acc: 93.385 | correct, total: (675673,723532)


 Eval--epochIndex 4 |time: 23 | Acc: 92.281 | correct, total: (1335372,1447064)
              precision    recall  f1-score   support

           0       0.97      0.95      0.96    549429
           1       0.74      0.90      0.82     86375
           2       0.77      0.56      0.65     60284
           3       0.67      0.86      0.75     27444

    accuracy                           0.91    723532
   macro avg       0.79      0.82      0.80    723532
weighted avg       0.92      0.91      0.91    723532

[[524049  13365   5420   6595]
 [  3791  78049   2121   2414]
 [ 1

##########one 20 batch totally time cost 0.482
batchIndex 1 |trainLen 142 | Loss: 0.164 | Acc: 93.184 | correct, total: (9542,10240)
##########one 20 batch totally time cost 3.805
batchIndex 21 |trainLen 142 | Loss: 0.156 | Acc: 93.390 | correct, total: (105194,112640)
##########one 20 batch totally time cost 7.226
batchIndex 41 |trainLen 142 | Loss: 0.154 | Acc: 93.447 | correct, total: (200948,215040)
##########one 20 batch totally time cost 10.548
batchIndex 61 |trainLen 142 | Loss: 0.157 | Acc: 93.319 | correct, total: (296233,317440)
##########one 20 batch totally time cost 13.838
batchIndex 81 |trainLen 142 | Loss: 0.157 | Acc: 93.295 | correct, total: (391689,419840)
##########one 20 batch totally time cost 17.208
batchIndex 101 |trainLen 142 | Loss: 0.156 | Acc: 93.334 | correct, total: (487429,522240)
##########one 20 batch totally time cost 20.524
batchIndex 121 |trainLen 142 | Loss: 0.156 | Acc: 93.359 | correct, total: (583155,624640)
##########one 20 batch totally time cos

##########one 20 batch totally time cost 24.670
batchIndex 141 |trainLen 142 | Loss: 0.155 | Acc: 93.352 | correct, total: (675428,723532)


 Eval--epochIndex 13 |time: 24 | Acc: 93.876 | correct, total: (1358442,1447064)
              precision    recall  f1-score   support

           0       0.96      1.00      0.98    549429
           1       0.85      0.90      0.87     86375
           2       0.97      0.57      0.72     60284
           3       0.83      0.89      0.86     27444

    accuracy                           0.94    723532
   macro avg       0.91      0.84      0.86    723532
weighted avg       0.95      0.94      0.94    723532

[[546886   2524     19      0]
 [  6563  77408    787   1617]
 [ 13244   9439  34382   3219]
 [  1585   1402    119  24338]]
########################################################################
##########one 20 batch totally time cost 0.482
batchIndex 1 |trainLen 142 | Loss: 0.153 | Acc: 93.643 | correct, total: (9589,10240)
##########on

In [5]:
print(tmp2)

[[542965   6462      2      0]
 [  5609  78786     53   1927]
 [ 12006  12810  33190   2278]
 [   809   1579   2335  22721]]


In [30]:
state = {"net": mlp.state_dict(), "optimizer": optimizer.state_dict()}
modelPathName = "./trainedModes/"+"redTLS1.modeparams"
torch.save(state, modelPathName)
params = torch.load(modelPathName)
mlp.load_state_dict(params['net'])
optimizer.load_state_dict(params['optimizer'])

In [48]:
tmp = y
trainratio = np.bincount(tmp.astype("int64"))
train_weights = 1./trainratio
print(trainratio),print(trainratio/sum(trainratio)*100),print(train_weights/sum(train_weights)*100)

train_sampleweights = torch.from_numpy(train_weights)
train_sampler = WeightedRandomSampler(weights=train_sampleweights, num_samples = 2*len(y))

[549429  86375  60284  27444]
[75.93706982 11.93796543  8.33190515  3.7930596 ]
[ 2.74010234 17.42971566 24.97332111 54.85686088]
